In [ ]:
config_dir = 'PATH'

## Imports

In [35]:
from hydra import compose, initialize_config_dir

with initialize_config_dir(config_dir=config_dir, version_base='1.3'):
        config_dict = compose(config_name='config')
    
import os
os.chdir(config_dict.cwd)

In [36]:
import json, numpy as np, pandas as pd, os
from typing import Dict, List
from proconsul.datasets_extraction.dataset_scheme import DatasetScheme

In [37]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [38]:
from time import gmtime, strftime

## Data

In [39]:
data_path = config_dict.path_after_llm_response
split = 'test'
with open(data_path, 'r') as file:
    points = [{DatasetScheme.ID_KEY: k} | v for k, v in json.load(file)[split].items()]
points = points[:1000]

## Labeling setup

In [40]:
q_to_labels = {
    'Is this summary Verbose?': config_dict.evaluation.semi_auto_labels['Verbosity'],
    'Is this summary Trivial?': config_dict.evaluation.semi_auto_labels['Triviality'],
    'Is this summary Factual?': config_dict.evaluation.semi_auto_labels['Factuality'],
    'Are there any Hallucinations?': config_dict.evaluation.semi_auto_labels['Hallucinations'],
}
    
all_widgets = {}
for q in q_to_labels:
    widget_axis = tuple([widgets.ToggleButtons(
        options=[('Unspecified', 'Unspecified'), ('Yes', 1), ('No', 0)],
        description=q,
    ) for _ in range(len(points))])
    all_widgets[q] = widget_axis


def screen_cur_labels(save_with_name,
                      all_widgets: Dict, 
                      points: List) -> None:
    for i in range(len(points)):
        for q, widget_axis in all_widgets.items():
            points[i][q_to_labels[q]] = widget_axis[i].value
    if save_with_name:
        filename = filename_.value
    else:
        filename = 'annotation_chkpt_'+strftime("%Y-%m-%d", gmtime())
    save_dir = ('/').join(config_dict.path_after_annotation.split('/')[:-1])

    full_path = f'{save_dir}/{filename}.json'
    old_data = dict()
    if os.path.exists(full_path):
        with open(full_path, 'r') as file:
            old_data = json.load(file)
    old_data[split] = {p[DatasetScheme.ID_KEY]: p for p in points}

    os.makedirs(save_dir, exist_ok=True)
    with open(f'{save_dir}/{filename}.json', 'w') as file:
        json.dump(old_data, file, indent=2)


save_state = False
next_state = False
slider = widgets.IntSlider(min=0, max=len(points)-1, step=1, value=0)
def get_studio(save_click: bool, 
               point_n: int,
               next_click: bool,
               all_widgets: Dict, 
               points: List) -> None:
    global save_state, next_state, slider
    if next_state != next_click:
        next_state = next_click
        slider.value += 1
    else:
        save_with_name = save_state!=save_click
        save_state = save_click
        screen_cur_labels(save_with_name, all_widgets, points)
        for q, widget_axis in all_widgets.items():
            display(widget_axis[point_n])
        print(f'______\n{points[point_n][config_dict.evaluation.llm_messages["fact_halluc_response"]]}')
        print(f'______\n{points[point_n][config_dict.evaluation.doc_to_eval_key]}')
        print(f'______\n{points[point_n][DatasetScheme.CODE_KEY]}')
        if config_dict.evaluation.seen_context_key is None:
            seen_cxt = ''
        else:
            seen_cxt = points[point_n][config_dict.evaluation.seen_context_key]
        print(f'\n______\nCONTEXT SEEN:\n\n{seen_cxt}')

In [41]:
default_filename = config_dict.path_after_annotation.split('/')[-1].split('.')[0]
default_dirname = config_dict.path_after_annotation.split('/')[-2]
name_ = widgets.Text(placeholder='used as dir name', 
                     description='Your name:',
                     value=default_dirname,
                    )
filename_ = widgets.Text(placeholder='filename to save', 
                         description='Filename:',
                         value=default_filename,
                        )

## Labeling itself

In [ ]:
print('Don\'t change 2 filds below if you want to use current evaluation config downstream.')
display(name_)
display(filename_)
print('\n')

interact(get_studio,
         save_click=widgets.ToggleButton(description='Click to Save',
                                         button_style='info',
                                         value=False),
         point_n=slider,
         next_click=widgets.ToggleButton(description='Next point',
                                         button_style='warning',
                                         value=False),
         all_widgets=fixed(all_widgets),
         points=fixed(points),
        );